# Full Adder
Below is a Logisim diagram of a Full Adder circuit implementation. 
![](./images/full_adder_logisim.png)

First we begin by importing magma. We will use the prefix `m` to distinguish between magma functions and native Python.

In [1]:
import os
os.environ['MANTLE_TARGET'] = 'ice40'

import magma as m
from mantle import And, Or, XOr

import mantle lattice ice40
import mantle lattice mantle40


We can define standard Python functions to instance, wire up inputs, and return the output of our logic gates.

In [2]:
def and2(a, b):
    # instance a 2-input And gate and wire up a and b
    return And(2)(a, b)

def or3(a, b, c):
    # instance a 3-input Or gate and wire up a, b, and c
    return Or(3)(a, b, c)

def xor3(a, b, c):
    # instance a 3-input XOr gate and wire up a, b, and c
    return XOr(3)(a, b, c)

Finally, we can define a circuit to implement a Full Adder using our previously defined logic gates.

In [3]:
class FullAdder(m.Circuit):
    name = "FullAdder"
    IO = ["a", m.In(m.Bit), "b", m.In(m.Bit), "cin", m.In(m.Bit), 
          "sum", m.Out(m.Bit), "cout", m.Out(m.Bit)]
    @classmethod
    def definition(io):
        # Generate the sum
        sum = xor3(io.a, io.b, io.cin)
        m.wire(sum, io.sum)
        # Generate the carry
        a_and_b = and2(io.a, io.b)
        b_and_cin = and2(io.b, io.cin)
        a_and_cin = and2(io.a, io.cin)
        cout = or3(a_and_b, b_and_cin, a_and_cin)
        m.wire(cout, io.cout)

We can import magma's verilog backend to view the verilog definition corresponding to our FullAdder circuit. Notice the logic gates defined in terms of the `SB_LUT4` primitive.

In [4]:
from magma.backend.verilog import compile as compile_verilog

print(compile_verilog(FullAdder))

compiling XOr3
compiling And2
compiling Or3
compiling FullAdder
module XOr3 (input [2:0] I, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h9696)) inst0 (.I0(I[0]), .I1(I[1]), .I2(I[2]), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module And2 (input [1:0] I, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h8888)) inst0 (.I0(I[0]), .I1(I[1]), .I2(1'b0), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module Or3 (input [2:0] I, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'hFEFE)) inst0 (.I0(I[0]), .I1(I[1]), .I2(I[2]), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module FullAdder (input  a, input  b, input  cin, output  sum, output  cout);
wire  inst0_O;
wire  inst1_O;
wire  inst2_O;
wire  inst3_O;
wire  inst4_O;
XOr3 inst0 (.I({cin,b,a}), .O(inst0_O));
And2 inst1 (.I({b,a}), .O(inst1_O));
And2 inst2 (.I({cin,b}), .O(inst2_O));
And2 inst3 (.I({cin,a}), .O(inst3_O));
Or3 inst4 (.I({inst3_O,inst2_O,inst1_O}), .O(inst4_O));
assign sum = inst0_O;
assi

We will test our circuit on the icestick by wiring the outputs to the `D1` and `D2` leds. We begin by importing the `IceStick` module from `loam`. With an instance of the `IceStick`, we turn on the clock, D1, and D2 pins. We then define a `main` function by calling `icestick.main()`, instancing our FullAdder, and wiring up the ports.

In [5]:
from loam.boards.icestick import IceStick

icestick = IceStick()

icestick.Clock.on()
icestick.D1.on()
icestick.D2.on()

main = icestick.main()
adder = FullAdder()
m.wire(0, adder.a)
m.wire(1, adder.b)
m.wire(1, adder.cin)
m.wire(adder.sum, main.D1)
m.wire(adder.cout, main.D2)

We use magma's `m.compile` function to generate verilog and pcf files for the icestick

In [6]:
m.compile("build/ice_full_adder", main)

compiling XOr3
compiling And2
compiling Or3
compiling FullAdder
compiling main


Finally, we use the yosys, arachne-pnr, and the icestorm tools to flash our circuit onto the icestick

In [7]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif ice_full_adder.blif' ice_full_adder.v
arachne-pnr -q -d 1k -o ice_full_adder.txt -p ice_full_adder.pcf ice_full_adder.blif
icepack ice_full_adder.txt ice_full_adder.bin

/Users/hanrahan/git/magma/notebooks/icestick/build


Uncomment the code below to program the IceStick board.

In [8]:
#!iceprog ice_full_adder.bin